In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

import unicodedata as ud
from sklearn.metrics import precision_recall_fscore_support as prfs
import csv
from itertools import izip
import HTMLParser
from sklearn.metrics import classification_report
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model

import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer as porterStemmer

D:\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_excel("C:\\Users\\swapnil sagar\\Documents\\UIC\\Data mining\\project 2\\romneyTweets_Filtered.xlsx");
data

,date,time,Anootated tweet,Class
0,NaN,NaN,"1: positive, -1: negative, 0: neutral, 2: mixed",NaN
1,2012-10-16 00:00:00,09:38:08-05:00,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1.0
2,2012-10-16 00:00:00,10:14:18-05:00,.@WardBrenda @shortwave8669 @allanbourdius you...,-1.0
3,2012-10-16 00:00:00,09:27:16-05:00,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1.0
4,2012-10-16 00:00:00,10:11:43-05:00,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1.0
5,2012-10-16 00:00:00,10:13:17-05:00,Hope <e>Romney</e> debate prepped w/ the same ...,1.0
6,2012-10-16 00:00:00,10:17:28-05:00,Want to know how <e>Mitt Romney</e> is going t...,-1.0
7,2012-10-16 00:00:00,09:35:55-05:00,If <e>Romney</e> wins the <a>presidential elec...,-1.0
8,2012-10-16 00:00:00,10:28:50-05:00,<e>Romney</e>'s <a>12 million jobs scam </a>re...,-1.0
9,2012-10-16 00:00:00,10:21:31-05:00,<e>Mitt #Romney</e> <a>said</a> that <a> catch...,-1.0


In [3]:
annotatedTweet = data['Anootated tweet']
actualClass = data['Class']
lengthObama = len(data);

annotatedTweet = annotatedTweet.drop(0)
actualClass = actualClass.drop(0)


annotatedTweetList = annotatedTweet.tolist();
actualClassList = actualClass.tolist();

In [4]:
count = 0;

finalFilteredTweet = []

def removeStopWords(tweet):
    filtered_tweet = [];
    
    stop_words = set(stopwords.words('english'));
    word_tokens = word_tokenize(tweet);
    
    for w in word_tokens:
        if w not in stop_words:
            filtered_tweet.append(w);
    
    eachTweet = " ".join(filtered_tweet)  
    return eachTweet
    
    
def preProcessTweets(annotatedTweet):
        
        #remove links
        processedTweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', annotatedTweet);
        #remove references with @<word>
        processedTweet = re.sub(r'(\s)@\w+', r'', processedTweet)
        processedTweet = re.sub(r'@\w+', r'', processedTweet)
        
        # remove HTML tags from tweet
        processedTweet = re.sub('<[^<]+?>', '', processedTweet)
        
        #remove exclamation marks - hashtags should be removed?
        processedTweet = re.sub(r'[<>!#@$:.,%\?-]+', r'', processedTweet)
        
        # remove extra white spaces
        processedTweet = re.sub(r'\s+', r' ', processedTweet)
        
        #remove "" 
        processedTweet = processedTweet.replace('"', '');
        
        processedTweet = ''.join([i if ord(i) < 128 else ' ' for i in processedTweet])

        #stemming
        stemmer = porterStemmer()
        stemmedTweet = [stemmer.stem(word) for word in processedTweet.split(" ")]
        stemmedTweet = " ".join(stemmedTweet)
        processedTweet = str(stemmedTweet);
        
        processedTweet = processedTweet.replace("'", "");

        #remove numbers from data
        #join
        #processedTweet = " ".join(re.findall('[A-Z][^A-Z]*', processedTweet));

        #processedTweet = removeStopWords(processedTweet); --- Remove stop words in the end
        
        return processedTweet;

    
for every_tweet in annotatedTweetList:
    
    count = count +1     
    tweet = preProcessTweets(every_tweet).encode('ascii', 'ignore').strip();
    #print type(tweet)

    finalFilteredTweet.append(tweet);        

In [5]:
len(finalFilteredTweet)

5648

In [6]:
finalFilteredTweet

['insidiousmitt romney bain help philip morri get US high schooler hook On cigarett via',
 'you mean like romney cheat in primari',
 'mitt romney still doesnt believ that we have a black presid',
 'romney tax plan deserv a 2nd look becaus he ha a secret one that differ than the one he been lie about',
 'hope romney debat prep w/ the same peopl as last time',
 'want to know how mitt romney is go to be abl to cut by 5 trillon dollar Go here it explain everyth',
 'If romney win the presidenti elect the world realli end thi year',
 'romney 12 million job scam remind me of rip torn sell penni in the movi sick puppi',
 'mitt romney said that catch osama bin laden would be insignific and it not worth move heaven and earth',
 'pleas be mitt romney   who is honey boo boo endors for presid (video)',
 'romney is disappoint when state allow samesex marriag',
 'If women that the poll show are truli vote for romney then I dont want to know how they truli take care of their household',
 'when it come

In [7]:
trainData, testTrainData , y_train, y_test = train_test_split(finalFilteredTweet, actualClassList, test_size = 0.15, random_state = 40)

In [8]:
vectorizer = TfidfVectorizer(min_df=0.00125,
                             max_df = 0.70,
                             sublinear_tf=True,
                             use_idf=True,
                            stop_words=u'english',
                            analyzer='word',
                            ngram_range=(1,6),lowercase=True)

In [17]:
train_vectors = vectorizer.fit_transform(trainData);
test_vectors = vectorizer.transform(testTrainData);
total_vectors = vectorizer.fit_transform(finalFilteredTweet);

knn = KNeighborsClassifier()

In [18]:
knn.fit(train_vectors, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [19]:
predicted_labels = knn.predict(test_vectors)
predicted_labelsList = predicted_labels.tolist();
knn.score(test_vectors, y_test)

0.45283018867924529

In [20]:
target_classifiers = ['-1', '0', '1']
print(classification_report(y_test, predicted_labelsList, target_names=target_classifiers))

             precision    recall  f1-score   support

         -1       0.56      0.49      0.52       425
          0       0.33      0.55      0.41       252
          1       0.61      0.23      0.33       171

avg / total       0.50      0.45      0.45       848



In [21]:
cvScores = cross_val_score(knn, total_vectors, actualClassList, cv=10,scoring='f1_weighted')
print (cvScores)

[ 0.46922076  0.42759434  0.44980694  0.43767091  0.46478542  0.50345161
  0.49352674  0.45189299  0.47819582  0.46653168]


In [22]:
print ("Mean: ", cvScores.mean())
print ("Minimum: ", cvScores.min())
print ("Maximum: ", cvScores.max())

('Mean: ', 0.46426772083821338)
('Minimum: ', 0.42759433539874225)
('Maximum: ', 0.50345161313214981)
